In [361]:
import pandas as pd
pd.options.display.max_colwidth = 200
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import light_location_dict
from Project._05InferKnowledgeOfRules.infer_rules_functions import json_to_dataframe

In [250]:
level1_df = json_to_dataframe(year=1,level=1)

In [251]:
level2_df = json_to_dataframe(year=1,level=2)

In [252]:
level3_df = json_to_dataframe(year=1,level=3)

In [253]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [307]:
def filter_rule_indexes(dataframe):
    level_3_check = len(re.findall('\*', dataframe.loc[0, 'pattern'])) > 0
    rule_type = ['app_app_app', 'psn_app_app', 'psn_psn_app']

    #filter follows rules in dataframe for level3: max 1 and level2: 0
    if level_3_check:
        return_dict = {
            'multi_floor': {'app_app_app': [],'psn_app_app': [],'psn_psn_app': []},
            'single_floor': {'app_app_app': [],'psn_app_app': [],'psn_psn_app': []}
        }
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) <= 1]
    else:
        return_dict = {
            'multi_floor': {'app_app': [],'psn_app': []},
            'single_floor': {'app_app': [],'psn_app': []}
        }
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) == 0]

    for index, row in dataframe.iterrows():
        tmp_floor_set = set()
        appliance_check_list = list()
        person_check_list = list()
        for col in set(re.findall('[\w_]+', row['pattern'])):
            person_check_list.append('SensHeat' in col)
            appliance_check_list.append('SensHeat' not in col)
            try:
                tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
            except:
                tmp_floor_set.add(light_location_dict(meta)[col][0])

        if sum(appliance_check_list) >= 1:
            if {'1stFloor', '2ndFloor'} == tmp_floor_set:
                if level_3_check:
                    return_dict['multi_floor'][rule_type[sum(person_check_list)]].append(index)
                else:
                    return_dict['multi_floor'][rule_type[sum(person_check_list)][:-4]].append(index)
            else:
                if level_3_check:
                    return_dict['single_floor'][rule_type[sum(person_check_list)]].append(index)
                else:
                    return_dict['single_floor'][rule_type[sum(person_check_list)][:-4]].append(index)

    return return_dict

In [308]:
level2_index_dict = filter_rule_indexes(level2_df)
level2_index_dict

{'multi_floor': {'app_app': [3,
   4,
   23,
   26,
   27,
   137,
   151,
   153,
   155,
   207,
   210,
   226,
   245,
   250,
   254,
   255,
   256,
   257,
   266,
   267,
   339,
   350,
   351,
   366,
   378,
   379,
   380,
   384,
   554,
   568,
   587,
   595,
   598,
   599,
   600,
   607,
   616,
   617,
   619,
   621,
   624,
   625,
   633,
   634,
   639,
   640,
   678,
   693],
  'psn_app': [2,
   33,
   38,
   44,
   51,
   54,
   69,
   70,
   156,
   213,
   253,
   264,
   265,
   269,
   270,
   274,
   278,
   282,
   283,
   284,
   293,
   294,
   301,
   302,
   334,
   341,
   343,
   368,
   370,
   389,
   390,
   392,
   408,
   413,
   417,
   482,
   483,
   485,
   487,
   489,
   496,
   502,
   504,
   505,
   508,
   510,
   511,
   520,
   521,
   523,
   526,
   530,
   542,
   544,
   545,
   584,
   588,
   589,
   608,
   609,
   623,
   630,
   631,
   635,
   636,
   647,
   648,
   650,
   652,
   654,
   661,
   667,
   668]},
 'single

In [272]:
level3_index_dict = filter_rule_indexes(level3_df)

In [310]:
def redundancy_filter_tool(dataframe=level3_df, regex_str='BR3[\w]*>[\w]*ChildB', select_index_dict='psn_app_app', multi_floor=True):
    multi_floor_selector = 'multi_floor' if multi_floor else 'single_floor'
    dataframe = dataframe.loc[filter_rule_indexes(dataframe)[multi_floor_selector][select_index_dict]]
    return dataframe.loc[dataframe['pattern'].str.findall(regex_str).map(len) > 0]

In [333]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', select_index_dict='psn_app', multi_floor=True)

,pattern,supp,conf,time
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:14:19'], ['2014-01-16 05:59:19', '2014-01-16 06..."
213,Load_StatusPlugLoadToaster>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'199': [[['2014-01-16 06:19:19', '2014-01-16 06:30:19'], ['2014-01-16 06:29:19', '2014-01-16 06..."
253,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntBUP,0.838356,0.838356,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:24:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
265,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildAUP,0.564384,0.564384,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 18:29:45', '2014-01-16 19..."
270,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 18:29:45', '2014-01-16 19..."
334,Load_StatusBR2Lights>Load_StatusSensHeatPrntADOWN,0.567123,0.571823,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 22..."
341,Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
343,Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
368,Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 20:29:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
370,Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 20:29:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."


In [263]:
redundancy_filter_tool(dataframe=level2_df, regex_str='SensHeat[\w]*>[\w]*', select_index_dict='psn_app', multi_floor=True)

,pattern,supp,conf,time
33,Load_StatusSensHeatPrntBDOWN>Load_StatusMBALights,0.991781,1.000000,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:29:19'], ['2014-01-16 05:59:19', '2014-01-16 06..."
38,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV,0.698630,0.704420,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:29:19'], ['2014-01-16 06:14:19', '2014-01-16 06..."
44,Load_StatusSensHeatPrntBDOWN>Load_StatusBA2Lights,0.983562,0.991713,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 20:14:45', '2014-01-16 20..."
51,Load_StatusSensHeatPrntBDOWN>Load_StatusBR2Lights,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 21..."
54,Load_StatusSensHeatPrntBDOWN>Load_StatusBR3Lights,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 20..."
69,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBR2Laptop,0.986301,0.994475,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 21..."
70,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBR3Laptop,0.846575,0.853591,"{'199': [[['2014-01-16 17:59:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 20..."
274,Load_StatusSensHeatPrntADOWN>Load_StatusMBALights,0.849315,0.856354,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
278,Load_StatusSensHeatPrntADOWN>Load_StatusBA2Lights,0.991781,1.000000,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
282,Load_StatusSensHeatPrntADOWN>Load_StatusBR2Lights,0.849315,0.856354,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:29:18'], ['2014-01-16 06:39:18', '2014-01-16 06..."


In [262]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*\|[\w]*', select_index_dict='psn_app', multi_floor=True)

,pattern,supp,conf,time
156,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN,0.701370,0.707182,"{'199': [[['2014-01-16 06:14:19', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
264,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
269,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildBUP,0.838356,0.838356,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
301,Load_StatusSensHeatPrntADOWN|Load_StatusPlugLoadFan,0.698630,0.704420,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 22:29:44'], ['2014-01-16 21:59:44', '2014-01-17 00..."
389,Load_StatusDRLights|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
390,Load_StatusDRLights|Load_StatusSensHeatChildBUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
417,Load_StatusSensHeatChildBDOWN|Load_StatusBR2Lights,0.558904,0.563536,"{'258': [[['2014-03-16 12:58:31', '2014-03-16 18:28:31'], ['2014-03-16 18:18:31', '2014-03-16 23..."
489,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadLRTV,0.975342,0.975342,"{'199': [[['2014-01-16 00:04:18', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
496,Load_StatusSensHeatChildAUP|Load_StatusDRLights,0.843836,0.843836,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 19:29:45'], ['2014-01-16 19:14:45', '2014-01-16 20..."
508,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadDesktopPCMonitor,0.698630,0.698630,"{'299': [[['2014-04-26 00:03:32', '2014-04-26 09:28:36'], ['2014-04-26 08:28:36', '2014-04-26 17..."


In [336]:
redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*\|[\w]*', select_index_dict='psn_app', multi_floor=True)

,pattern,supp,conf,time
156,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN,0.701370,0.707182,"{'199': [[['2014-01-16 06:14:19', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
264,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
269,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildBUP,0.838356,0.838356,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
301,Load_StatusSensHeatPrntADOWN|Load_StatusPlugLoadFan,0.698630,0.704420,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 22:29:44'], ['2014-01-16 21:59:44', '2014-01-17 00..."
389,Load_StatusDRLights|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
390,Load_StatusDRLights|Load_StatusSensHeatChildBUP,0.701370,0.701370,"{'199': [[['2014-01-16 19:14:45', '2014-01-16 20:29:44'], ['2014-01-16 20:14:45', '2014-01-16 23..."
417,Load_StatusSensHeatChildBDOWN|Load_StatusBR2Lights,0.558904,0.563536,"{'258': [[['2014-03-16 12:58:31', '2014-03-16 18:28:31'], ['2014-03-16 18:18:31', '2014-03-16 23..."
489,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadLRTV,0.975342,0.975342,"{'199': [[['2014-01-16 00:04:18', '2014-01-16 06:59:18'], ['2014-01-16 06:29:19', '2014-01-16 08..."
496,Load_StatusSensHeatChildAUP|Load_StatusDRLights,0.843836,0.843836,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 19:29:45'], ['2014-01-16 19:14:45', '2014-01-16 20..."
508,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadDesktopPCMonitor,0.698630,0.698630,"{'299': [[['2014-04-26 00:03:32', '2014-04-26 09:28:36'], ['2014-04-26 08:28:36', '2014-04-26 17..."


In [334]:
def potential_redundant_time_intervals(dataframe):
    rule_potential_redundancy = {}
    for index, row in psn_app_df.iterrows():
        rule_potential_redundancy[row['pattern']] = []
        for time_intervals in row['time'].values():
            for time_interval in time_intervals:
                rule_potential_redundancy[row['pattern']].append(time_interval[1])
    return rule_potential_redundancy

In [344]:
psn_app_df = redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', select_index_dict='psn_app', multi_floor=True)
psn_app_df.loc[~psn_app_df['pattern'].str.contains('Toaster')]

,pattern,supp,conf,time
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 05:59:19', '2014-01-16 06:14:19'], ['2014-01-16 05:59:19', '2014-01-16 06..."
253,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntBUP,0.838356,0.838356,"{'199': [[['2014-01-16 06:29:19', '2014-01-16 08:24:18'], ['2014-01-16 06:29:19', '2014-01-16 06..."
265,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildAUP,0.564384,0.564384,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 18:29:45', '2014-01-16 19..."
270,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'199': [[['2014-01-16 18:09:45', '2014-01-16 20:29:44'], ['2014-01-16 18:29:45', '2014-01-16 19..."
334,Load_StatusBR2Lights>Load_StatusSensHeatPrntADOWN,0.567123,0.571823,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 22..."
341,Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
343,Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
368,Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 20:29:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
370,Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 20:29:44'], ['2014-01-16 19:29:45', '2014-01-16 20..."
584,Load_StatusPlugLoadBR2Laptop>Load_StatusSensHeatPrntADOWN,0.706849,0.712707,"{'199': [[['2014-01-16 18:29:45', '2014-01-16 21:59:44'], ['2014-01-16 18:29:45', '2014-01-16 22..."


In [345]:
potential_redundant_time_intervals(psn_app_df)

{'Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN': [['2014-01-16 05:59:19',
   '2014-01-16 06:29:19'],
  ['2013-10-17 05:59:51', '2013-10-17 06:29:51'],
  ['2014-04-08 05:58:53', '2014-04-08 06:28:53'],
  ['2013-12-26 06:00:01', '2013-12-26 06:30:01'],
  ['2014-02-24 05:58:13', '2014-02-24 06:28:13'],
  ['2014-06-18 05:58:42', '2014-06-18 06:28:42'],
  ['2014-06-12 05:58:25', '2014-06-12 06:28:25'],
  ['2013-10-01 05:59:38', '2013-10-01 06:29:38'],
  ['2014-02-19 05:58:55', '2014-02-19 06:28:55'],
  ['2014-04-24 05:58:10', '2014-04-24 06:28:10'],
  ['2013-07-31 05:59:50', '2013-07-31 06:29:50'],
  ['2013-11-20 05:59:11', '2013-11-20 06:29:11'],
  ['2013-08-12 05:59:11', '2013-08-12 06:29:11'],
  ['2013-10-15 05:59:52', '2013-10-15 06:29:52'],
  ['2013-09-13 05:59:38', '2013-09-13 06:29:38'],
  ['2014-03-14 05:58:23', '2014-03-14 06:28:23'],
  ['2013-07-05 05:58:29', '2013-07-05 06:28:29'],
  ['2013-08-07 05:59:15', '2013-08-07 06:29:15'],
  ['2013-10-08 05:59:55', '2013-10-08 06:29:

In [348]:
psn_app_app_df = redundancy_filter_tool(dataframe=level3_df, regex_str='[\w]*>[\w]*SensHeat', select_index_dict='psn_app_app', multi_floor=True)

In [362]:
psn_app_app_df

,pattern,supp,conf,time
6,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBlender*Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:04:11', '2013-11-21 06:12:11']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
7,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadCoffeeMaker*Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:09:11', '2013-11-21 06:27:11']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
8,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV*Load_StatusMBALights->Load_StatusPlugLoadMBRTV,0.698630,0.704420,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:14:11', '2013-11-21 06:59:10']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
9,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadToasterOven*Load_StatusMBALights->Load_StatusPlugLoadToasterOven,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:14:11', '2013-11-21 06:24:11']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
12,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadToaster*Load_StatusMBALights->Load_StatusPlugLoadToaster,0.701370,0.707182,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:14:11'], ['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:19:11', '2013-11-21 06:30:11']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
...,...,...,...,...
4733,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP|Load_StatusDRLights*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusDRLights,0.701370,0.707182,"{'143': [[['2013-11-21 18:29:10', '2013-11-21 22:23:09'], ['2013-11-21 18:29:10', '2013-11-21 19:29:10'], ['2013-11-21 19:14:10', '2013-11-21 20:29:09']]], '102': [[['2013-10-11 18:29:55', '2013-1..."
4738,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP>Load_StatusPlugLoadCanOpener*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusPlugLoadCanOpener,0.726027,0.734072,"{'143': [[['2013-11-21 18:29:10', '2013-11-21 22:23:09'], ['2013-11-21 18:29:10', '2013-11-21 19:29:10'], ['2013-11-21 18:39:10', '2013-11-21 18:46:10']]], '102': [[['2013-10-11 18:29:55', '2013-1..."
4739,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP>Load_StatusPlugLoadHandMixer*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusPlugLoadHandMixer,0.726027,0.734072,"{'143': [[['2013-11-21 18:29:10', '2013-11-21 22:23:09'], ['2013-11-21 18:29:10', '2013-11-21 19:29:10'], ['2013-11-21 18:39:10', '2013-11-21 18:41:10']]], '102': [[['2013-10-11 18:29:55', '2013-1..."
4751,Load_StatusPlugLoadDesktopPCMonitor|Load_StatusPlugLoadFan*Load_StatusPlugLoadFan>Load_StatusSensHeatPrntBUP*Load_StatusPlugLoadDesktopPCMonitor|Load_StatusSensHeatPrntBUP,0.547945,0.547945,"{'143': [[['2013-11-21 18:29:10', '2013-11-21 22:23:09'], ['2013-11-21 21:59:09', '2013-11-22 00:00:09'], ['2013-11-21 21:59:09', '2013-11-21 23:58:09']]], '199': [[['2014-01-16 18:29:45', '2014-0..."


In [368]:
psn_psn_app_df = redundancy_filter_tool(dataframe=level3_df, regex_str='MBR[\w]*>[\w]*Prnt', select_index_dict='psn_psn_app', multi_floor=True)

In [369]:
psn_psn_app_df

,pattern,supp,conf,time
349,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV*Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP*Load_StatusSensHeatPrntBDOWN->Load_StatusSensHeatPrntBUP,0.69863,0.69863,"{'143': [[['2013-11-21 05:59:11', '2013-11-21 06:29:11'], ['2013-11-21 06:14:11', '2013-11-21 06:59:10'], ['2013-11-21 06:29:11', '2013-11-21 06:59:10']]], '102': [[['2013-10-11 05:59:56', '2013-1..."
1499,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN*Load_StatusSensHeatPrntADOWN>Load_StatusSensHeatPrntBUP*Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP,0.70137,0.70137,"{'143': [[['2013-11-21 06:14:11', '2013-11-21 06:59:10'], ['2013-11-21 06:29:11', '2013-11-21 08:29:10'], ['2013-11-21 06:29:11', '2013-11-21 06:59:10']]], '102': [[['2013-10-11 06:14:56', '2013-1..."


In [ ]:
level3_df.loc[level3_df['pattern'].str.contains('VideoGame')]

# Creating a list of redundant rules
## First, look at ownership in level2 for same floor

## Second, look at rules in level3 that could be optimised and append to list